# Modul 8 : Ekstraksi Ciri/Fitur

In [131]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import math

### Gray Level Co-Occurence Matrix (GlCM)

In [119]:
matriks = np.array([
    [0,0,1,1],
    [0,0,1,1],
    [0,2,2,2],
    [2,2,3,3],
])
print(matriks)

[[0 0 1 1]
 [0 0 1 1]
 [0 2 2 2]
 [2 2 3 3]]


##### Memeriksa suatu indeks pada suatu matriks out of bounds atau tidak

In [120]:
def is_index_valid(arr, i, j):
    return 0 <= i < len(arr) and 0 <= j < len(arr[0])

##### Melakukan Proses Normalisasi dengan membagi setiap nilai piksel dengan jumlah total keseluruhan piksel dari suatu matriks

In [121]:
def normalisasi(image):
    sum = np.sum(image)
    result = np.zeros(image.shape)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            result[i,j] = image[i][j]/sum
    return result

##### Menghitung Matriks Konkurensi Simetris dengan sudut 0 derajat

In [122]:
def derajat0(image, bit):
    result = np.zeros((bit**2, bit**2))
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if is_index_valid(image,i,j+1):
                result[image[i,j], image[i,j+1]] += 1
    result = result + result.T
    result = normalisasi(result)
    return result
tetha0 = derajat0(matriks, 2)
print(tetha0)

[[0.16666667 0.08333333 0.04166667 0.        ]
 [0.08333333 0.16666667 0.         0.        ]
 [0.04166667 0.         0.25       0.04166667]
 [0.         0.         0.04166667 0.08333333]]


##### Menghitung Matriks Konkurensi Simetris dengan sudut 90 derajat

In [123]:
def derajat90(image, bit):
    result = np.zeros((bit**2, bit**2))
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if is_index_valid(image,i-1,j):
                result[image[i,j], image[i-1,j]] += 1
    result = result + result.T
    result = normalisasi(result)
    return result
tetha90 = derajat90(matriks, 2)
print(tetha90)

[[0.25       0.         0.08333333 0.        ]
 [0.         0.16666667 0.08333333 0.        ]
 [0.08333333 0.08333333 0.08333333 0.08333333]
 [0.         0.         0.08333333 0.        ]]


##### Menghitung Matriks Konkurensi Simetris dengan sudut 45 derajat

In [124]:
def derajat45(image, bit):
    result = np.zeros((bit**2, bit**2))
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if is_index_valid(image,i-1,j+1):
                result[image[i,j], image[i-1,j+1]] += 1
    result = result + result.T
    result = normalisasi(result)
    return result
tetha45 = derajat45(matriks, 2)
print(tetha45)

[[0.22222222 0.05555556 0.         0.        ]
 [0.05555556 0.11111111 0.11111111 0.        ]
 [0.         0.11111111 0.22222222 0.05555556]
 [0.         0.         0.05555556 0.        ]]


##### Menghitung Matriks Konkurensi Simetris dengan sudut 135 derajat

In [125]:
def derajat135(image, bit):
    result = np.zeros((bit**2, bit**2))
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if is_index_valid(image,i-1,j-1):
                result[image[i,j], image[i-1,j-1]] += 1
    result = result + result.T
    result = normalisasi(result)
    return result
tetha135 = derajat135(matriks, 2)
print(tetha135)

[[0.11111111 0.05555556 0.16666667 0.        ]
 [0.05555556 0.11111111 0.05555556 0.        ]
 [0.16666667 0.05555556 0.         0.11111111]
 [0.         0.         0.11111111 0.        ]]


##### Menghitung Angular Second Moment (ASM)

In [126]:
def ASM(matriks):
    asm = np.sum(tetha0**2)
    return asm
asmMat = ASM(tetha0)
print(asmMat)

0.14583333333333331


##### Menghitung Contrast

In [127]:
def contrast(matriks):
    result = 0
    for i in range(matriks.shape[0]):
        for j in range(matriks.shape[1]):
            result+=((i-j)**2)*matriks[i,j]
    return result
con = contrast(tetha0)
print(con)

0.5833333333333333


##### Menghitung Correlation

###### mencari rata rata intensitas piksel

In [128]:
def idxMiu(matriks):
    miuI = 0
    miuJ = 0
    for i in range(matriks.shape[0]):
        for j in range(matriks.shape[1]):
            miuI += i*matriks[i,j]
            miuJ += j*matriks[i,j]
    
    return miuI, miuJ
print(idxMiu(tetha0))


(1.2916666666666665, 1.2916666666666665)


###### Mencari standar devisiasi

In [129]:
def stdDev(matriks):
    miuI, miuJ = idxMiu(matriks)
    stdDevI = 0
    stdDevJ = 0
    for i in range(matriks.shape[0]):
        for j in range(matriks.shape[1]):
            stdDevI += np.sqrt(((i-miuI)**2)*matriks[i,j])
            stdDevJ += np.sqrt(((j-miuJ)**2)*matriks[i,j])
    return stdDevI, stdDevJ
print(stdDev(tetha0))

(2.852330419383406, 2.8523304193834065)


In [130]:
def correlation(matriks):
    corr = 0 
    miuI, miuJ = idxMiu(matriks)
    stdDevI, stdDevJ = stdDev(matriks)
    for i in range(matriks.shape[0]):
        for j in range(matriks.shape[1]):
            corr += ((i-miuI)*(j-miuJ)*matriks[i,j])/(stdDevI*stdDevJ)
    return corr
print(correlation(tetha0))

0.09197189176373607


##### Menghitung Entropy

In [137]:
def entropy(matriks):
    ent = 0
    for i in range(matriks.shape[0]):
        for j in range(matriks.shape[1]):
            if matriks[i,j] != 0:
                ent -= matriks[i,j]*np.log2(matriks[i,j])
    return ent
print(entropy(tetha0))

3.0220552088742


##### Menghitung Energy

In [138]:
def energy(matriks):
    return np.sqrt(ASM(matriks))
print(energy(tetha0))

0.38188130791298663


##### Menghitung Homogenitas

In [141]:
def homogenity(matriks):
    hom = 0
    for i in range(matriks.shape[0]):
        for j in range(matriks.shape[1]):
            hom += matriks[i,j]/(1+(abs(i-j)))
    return hom
print(homogenity(tetha0))

0.8194444444444445


##### Menghitung Disimilaritas

In [142]:
def dissimilarity(matriks):
    dis = 0
    for i in range(matriks.shape[0]):
        for j in range(matriks.shape[1]):
            dis += matriks[i,j]*abs(i-j)
    return dis
print(dissimilarity(tetha0))

0.4166666666666667
